In [1]:
from queue import PriorityQueue
import heapq


In [2]:
def turn_left(dir):
  if dir == "^":
    return "<"
  if dir == "<":
    return "v"
  if dir == "v":
    return ">"
  if dir == ">":
    return "^"
  return dir

In [3]:
def turn_right(dir):
  if dir == "^":
    return ">"
  if dir == ">":
    return "v"
  if dir == "v":
    return "<"
  if dir == "<":
    return "^"
  return dir

In [4]:
def walk(pos, dir):
  (x, y) = pos
  if dir == "^":
    return (x, y-1)
  if dir == "v":
    return (x, y+1)
  if dir == "<":
    return (x-1, y)
  if dir == ">":
    return (x+1, y)
  return pos

In [5]:
# def get_cost(start_orientation, end_pos, barriers):
#   investigate = dict()
#   seen = set()

#   investigate.update([(start_orientation, 0)])
#   seen.add(start_orientation)

#   while investigate:
#     print(investigate)
#     (pos, dir) = min(investigate, key=investigate.get)
#     cost = investigate.pop((pos, dir))
#     seen.add((pos, dir))

#     l_dir = turn_left(dir)
#     r_dir = turn_right(dir)
#     new_pos = walk(pos, dir)

#     if new_pos == end_pos:
#       return cost + 1
    
#     if not (new_pos, dir) in seen and not new_pos in barriers:
#       investigate.update([((new_pos, dir), cost + 1)])

#     if not (pos, l_dir) in seen:
#       investigate.update([((pos, l_dir), cost + 1000)])

#     if not (pos, r_dir) in seen:
#       investigate.update([((pos, r_dir), cost + 1000)])

#   return -1
  

In [6]:
# def get_cost(start_orientation, end_pos, barriers):
#   investigate = PriorityQueue()
#   seen = set()

#   investigate.put((0, start_orientation))
#   seen.add(start_orientation)

#   while investigate:
#     cost, (pos, dir) = investigate.get()
#     seen.add((pos, dir))

#     l_dir = turn_left(dir)
#     r_dir = turn_right(dir)
#     new_pos = walk(pos, dir)

#     if new_pos == end_pos:
#       return cost + 1
    
#     if not (new_pos, dir) in seen and not new_pos in barriers:
#       investigate.put((cost + 1, (new_pos, dir)))

#     if not (pos, l_dir) in seen:
#       investigate.put((cost + 1000, (pos, l_dir)))

#     if not (pos, r_dir) in seen:
#       investigate.put((cost + 1000, (pos, r_dir)))

#   return -1
  

In [7]:
# def get_cost(start_orientation, end_pos, barriers):
#   investigate = []
#   seen = set()

#   heapq.heappush(investigate, (0, start_orientation))
#   seen.add(start_orientation)

#   while investigate:

#     print(investigate)
#     cost, (pos, dir) = heapq.heappop(investigate)
#     seen.add((pos, dir))

#     l_dir = turn_left(dir)
#     r_dir = turn_right(dir)
#     new_pos = walk(pos, dir)

#     if new_pos == end_pos:
#       return cost + 1
    
#     if not (new_pos, dir) in seen and not new_pos in barriers:
#       heapq.heappush(investigate, (cost + 1, (new_pos, dir)))

#     if not (pos, l_dir) in seen:
#       heapq.heappush(investigate, (cost + 1000, (pos, l_dir)))

#     if not (pos, r_dir) in seen:
#       heapq.heappush(investigate, (cost + 1000, (pos, r_dir)))

#   return -1
  

In [8]:
# def get_path(start_orientation, end_pos, barriers):
#   investigate = PriorityQueue()
#   seen = set()

#   investigate.put((0, (start_orientation, [start_orientation])))
#   seen.add(start_orientation)

#   while investigate:
#     cost, ((pos, dir), path) = investigate.get()

#     l_dir = turn_left(dir)
#     r_dir = turn_right(dir)
#     new_pos = walk(pos, dir)

#     if new_pos == end_pos:
#       return cost + 1, path
    
#     if not (new_pos, dir) in seen and not new_pos in barriers:
#       investigate.put((cost + 1, ((new_pos, dir), path + [(new_pos, dir)])))
#       seen.add((new_pos, dir))

#     if not (pos, l_dir) in seen:
#       investigate.put((cost + 1000, ((pos, l_dir), path + [(pos, l_dir)])))
#       seen.add((pos, l_dir))

#     if not (pos, r_dir) in seen:
#       investigate.put((cost + 1000, ((pos, r_dir), path + [(pos, r_dir)])))
#       seen.add((pos, r_dir))

#   return -1, []
  

In [9]:
# Idea: paths can no longer be attached to the priority queue enteries because it cannot be updated at arbitrary times
# So we can try to keep a parallel dict that just keeps track of that (since the orientations are unique)

def get_paths(start_orientation, end_pos, barriers):
  investigate = PriorityQueue()
  seen = set()
  seen_costs = dict()
  seen_paths = dict()

  investigate.put((0, start_orientation))
  seen_costs.update({start_orientation: 0})
  seen_paths.update({start_orientation: {start_orientation}})

  while investigate:
    cost, orientation = investigate.get()
    (pos, dir) = orientation
    if pos == end_pos:
      return seen_costs[orientation], seen_paths[orientation]

    l_dir = turn_left(dir)
    r_dir = turn_right(dir)
    new_pos = walk(pos, dir)

    new_orientations = (((new_pos, dir), 1), 
                        ((pos, l_dir), 1000),
                        ((pos, r_dir), 1000))
    
    for new_orientation, new_cost in new_orientations:
      if not new_orientation in seen and not new_orientation[0] in barriers:
        investigate.put((cost + new_cost, new_orientation))
        seen.add(new_orientation)
        seen_costs.update({new_orientation: cost + new_cost})
        seen_paths.update({new_orientation: seen_paths[orientation] | {new_orientation}})
      elif new_orientation in seen and seen_costs[new_orientation] == cost + new_cost: 
        seen_paths.update({new_orientation: seen_paths[new_orientation] | seen_paths[orientation]})

  return -1, set()

In [10]:
# def is_on_path(orientation, cost_incurred, end_pos, barriers, total_cost, found_in_path, found_not_in_path, seen):

#   if orientation in found_in_path:
#     if found_in_path[orientation] == cost_incurred:
#       return True
#     else:
#       return False
  
#   if orientation in found_not_in_path:
#     if found_not_in_path[orientation] <= cost_incurred:
#       found_not_in_path.update([(orientation, cost_incurred)])
#       return False

#   (pos, dir) = orientation

#   if orientation in seen:
#     if seen[orientation] < cost_incurred:
#       found_not_in_path.update([(orientation, cost_incurred)])
#       return False
    
#   seen.update([(orientation, cost_incurred)])

#   if pos in barriers:
#     found_not_in_path.update([(orientation, cost_incurred)])
#     return False

#   if cost_incurred >= total_cost:
#     found_not_in_path.update([(orientation, cost_incurred)])
#     return False
  
#   # print(pos, dir, cost_incurred)

#   l_dir = turn_left(dir)
#   r_dir = turn_right(dir)
#   new_pos = walk(pos, dir)

#   a = is_on_path((new_pos, dir), cost_incurred + 1, end_pos, barriers, total_cost, found_in_path, found_not_in_path, seen)
#   b = is_on_path((pos, l_dir), cost_incurred + 1000, end_pos, barriers, total_cost, found_in_path, found_not_in_path, seen)
#   c = is_on_path((pos, r_dir), cost_incurred + 1000, end_pos, barriers, total_cost, found_in_path, found_not_in_path, seen)
#   if a or b or c:
#     found_in_path.update([(orientation, cost_incurred)])
#     return True
  
#   found_not_in_path.update([(orientation, cost_incurred)])
#   return False


In [11]:
f = open("input.txt")
input = f.readlines()

barriers = set()

for y, rows in enumerate(input):
  for x, sym in enumerate(rows):
    if sym == "#":
      barriers.add((x, y))
    elif sym == "S":
      start_pos = (x, y)
    elif sym == "E":
      end_pos = (x, y)

In [12]:
# cost = get_cost((start_pos, ">"), end_pos, barriers)
# cost

In [13]:
# cost, path = get_path((start_pos, ">"), end_pos, barriers)
# cost

In [14]:
cost, paths = get_paths((start_pos, ">"), end_pos, barriers)

unique_pos = set()
for item in paths:
  (pos, dir) = item
  unique_pos.add(pos)

print(cost)

print(len(unique_pos))

90460
575


In [15]:
# found = dict()
# found.update([((end_pos, "^"), cost)])
# found.update([((end_pos, "v"), cost)])
# found.update([((end_pos, ">"), cost)])
# found.update([((end_pos, "<"), cost)])

# is_on_path((start_pos, ">"), 0, end_pos, barriers, cost, found, dict(), dict())
  

In [16]:
# unique_pos = set()
# for item in found.keys():
#   (pos, dir) = item
#   unique_pos.add(pos)

# len(unique_pos)

In [17]:
scene = [[" " for _ in range(len(input[0]) - 1)] for _ in range(len(input))]

for pos in barriers:
  (x, y) = pos
  scene[y][x] = "#"

for pos in unique_pos:
  (x, y) = pos
  scene[y][x] = "."

str = ""
for row in scene:
  for sym in row:
    str += sym
  str += '\n'
print(str)

#############################################################################################################################################
#.......#...# #        .................#       #   #           #   #     #  ...........#             #       #     #     #          .......#
#.#####.#.#.# # #######.#####.###.#.#.#.# # ### # # ##### ### # ### # ### # #.### #####.##### ####### # ##### # # ### # # ####### ###.##### #
#.    #...#.#      ...................#.# # # #   #       #     #   # # #   #.    #   #.............#   #   #   #     # #   #   #    .    # #
#.### #####.#######.##### # ### #######.# # # ############# # ### ### # #####.####### #######.#####.# ### # ##### ##### ### # # # # #.### # #
#.# #   # #.........#     #   # #.......# # # #   #     #   #     #   #     #.  #       #   #.................#         # #   # # #  .#   # #
#.# ### # ########### ####### # #.####### # # # # # ### # ### ### # ### ### #.# # ##### # # # # # # ### #####.# # ####### ##### # ###.##### #
#.#   

In [18]:
bin(2067859)

'0b111111000110110010011'

In [19]:
111111000110110010111

111111000110110010111

In [20]:
found.keys()

NameError: name 'found' is not defined

In [813]:
def count_steps_dyn(num, seen):
  if num in seen:
    return seen[num]
  
  if num == 1 or num == 0:
      return 1

  steps = count_steps_dyn(num - 1, seen) + count_steps_dyn(num - 2, seen)
  seen.update({num: steps})
  
  return steps

In [814]:
def count_steps(num):
  
  if num == 1 or num == 0:
      return 1

  return count_steps(num - 2) + count_steps(num - 1)


In [815]:
count_steps_dyn(2000, dict())

6835702259575806647045396549170580107055408029365524565407553367798082454408054014954534318953113802726603726769523447478238192192714526677939943338306101405105414819705664090901813637296453767095528104868264704914433529355579148731044685634135487735897954629842516947101494253575869699893400976539545740214819819151952085089538422954565146720383752121972115725761141759114990448978941370030912401573418221496592822626

In [816]:
# count_steps(40)

In [817]:
unique_pos

{(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 131),
 (1, 132),
 (1, 133),
 (1, 134),
 (1, 135),
 (1, 136),
 (1, 137),
 (1, 138),
 (1, 139),
 (2, 1),
 (2, 17),
 (2, 131),
 (3, 1),
 (3, 17),
 (3, 18),
 (3, 19),
 (3, 99),
 (3, 100),
 (3, 101),
 (3, 102),
 (3, 103),
 (3, 131),
 (4, 1),
 (4, 19),
 (4, 99),
 (4, 103),
 (4, 131),
 (5, 1),
 (5, 19),
 (5, 49),
 (5, 50),
 (5, 51),
 (5, 52),
 (5, 53),
 (5, 54),
 (5, 55),
 (5, 56),
 (5, 57),
 (5, 58),
 (5, 59),
 (5, 60),
 (5, 61),
 (5, 62),
 (5, 63),
 (5, 64),
 (5, 65),
 (5, 66),
 (5, 67),
 (5, 68),
 (5, 69),
 (5, 99),
 (5, 103),
 (5, 105),
 (5, 106),
 (5, 107),
 (5, 117),
 (5, 118),
 (5, 119),
 (5, 120),
 (5, 121),
 (5, 122),
 (5, 123),
 (5, 124),
 (5, 125),
 (5, 126),
 (5, 127),
 (5, 128),
 (5, 129),
 (5, 130),
 (5, 131),
 (6, 1),
 (6, 19),
 (6, 49),
 (6, 69),
 (6, 99),
 (6, 103),
 (6, 105),
 (6, 107),
 (6, 117),
 (7, 1),
 (7, 

In [818]:
unique_pos = set()
for item in found.keys():
  print(item)

((139, 1), '^')
((139, 1), 'v')
((139, 1), '>')
((139, 1), '<')
((138, 1), '>')
((137, 1), '>')
((136, 1), '>')
((135, 1), '>')
((134, 1), '>')
((133, 1), '>')
((133, 1), '^')
((133, 2), '^')
((133, 3), '^')
((133, 4), '^')
((133, 5), '^')
((133, 6), '^')
((133, 7), '^')
((133, 8), '^')
((133, 9), '^')
((133, 10), '^')
((133, 11), '^')
((133, 12), '^')
((133, 13), '^')
((133, 13), '>')
((132, 13), '>')
((131, 13), '>')
((130, 13), '>')
((129, 13), '>')
((128, 13), '>')
((127, 13), '>')
((126, 13), '>')
((125, 13), '>')
((125, 13), 'v')
((125, 12), 'v')
((125, 11), 'v')
((125, 10), 'v')
((125, 9), 'v')
((125, 8), 'v')
((125, 7), 'v')
((125, 7), '>')
((124, 7), '>')
((123, 7), '>')
((122, 7), '>')
((121, 7), '>')
((120, 7), '>')
((119, 7), '>')
((118, 7), '>')
((117, 7), '>')
((116, 7), '>')
((115, 7), '>')
((114, 7), '>')
((124, 13), '>')
((123, 13), '>')
((122, 13), '>')
((121, 13), '>')
((120, 13), '>')
((119, 13), '>')
((118, 13), '>')
((117, 13), '>')
((116, 13), '>')
((115, 13), '>